In [3]:
import tensorflow as tf
import keras
from keras import Model, ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications import ResNet50
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import time
import os

CWD = os.getcwd()
os.chdir(os.pardir)
ROOT = os.getcwd()
print(CWD)
print(ROOT)

start_time = time.localtime(time.time())

/Users/tyleryang/Code/Polished-Capstone/tests
/Users/tyleryang/Code/Polished-Capstone


In [ ]:
image_size = 224
train_batch_size = 10
val_batch_size = 10
test_batch_size = 10

In [ ]:
valid_path = f'{ROOT}/datasets/ISIC-20' # validation dir
num_val_samples = 397  # ISIC-20 validation subset
val_steps = np.ceil(num_val_samples / val_batch_size)

valid_batches = ImageDataGenerator(
    tf.keras.applications.resnet50.preprocess_input
).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size,
    class_mode="categorical",
)

In [ ]:
test_path = f'{ROOT}/datasets/REZK' # test dir
num_test_samples = 607  # Dermatlas test set
test_steps = np.ceil(num_test_samples / test_batch_size)

test_batches = ImageDataGenerator(
    tf.keras.applications.resnet50.preprocess_input
).flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    batch_size=test_batch_size,
    class_mode="categorical",
    shuffle=False,
)

In [1]:
train_num = input("Which training dataset to use? Enter '1', '2', or '3")
print(f'You have selected train dataset{train_num}')

if train_num == 1:
    train_path = f'{ROOT}/data/train/dataset1'
    filepath = f'{CWD}/checkpoints/checkpoint_filepath_M1'
    num_train_samples = 1583
elif train_num == 2:
    train_path = f'{ROOT}/datasets/DiDI'
    filepath = f'{CWD}/checkpoints/checkpoint_filepath_M2'
    num_train_samples = 656
elif train_num == 3:
    train_path = f'{ROOT}/datasets/ISIC-80'
    filepath = '{CWD}/checkpoints/checkpoint_filepath_M3'
    num_train_samples = 2111

train_steps = np.ceil(num_train_samples / train_batch_size)

train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=train_batch_size,
    class_mode="categorical",
)

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# freezing all blocks except the last
for layer in base_model.layers[0:143]:
    layer.trainable = False

for layer in base_model.layers[143:]:
    layer.trainable = True

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

checkpoint = ModelCheckpoint(
    filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"
)

# try early stopping to avoid overfitting

earlystopping = EarlyStopping(
    monitor="val_accuracy", mode="max", patience=5, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.5,
    patience=5,
    verbose=1,
    mode="max",
    min_lr=0.00001,
)

callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(
    train_batches,
    steps_per_epoch=train_steps,
    validation_data=valid_batches,
    validation_steps=val_steps,
    epochs=30,  # was 20 but initial 30
    verbose=1,
    callbacks=callbacks_list,
)

model.save(f'{ROOT}/models/ResNet_M{train_num}.keras')

In [ ]:
### Model Training Metadata ###

end_time = time.localtime(time.time())

f = open(f"{CWD}/models/MODEL_INFO.txt", "a")
f.write(f'train dataset{train_num}\n')
f.write(f"Start Time: {start_time.tm_year}/{start_time.tm_mon}/{start_time.tm_mday}, {start_time.tm_hour}:{start_time.tm_min}:{start_time.tm_sec}\n")
f.write(f"End Time: {end_time.tm_year}/{end_time.tm_mon}/{end_time.tm_mday}, {end_time.tm_hour}:{end_time.tm_min}:{end_time.tm_sec}\n")
f.write('\n')
f.write(f'Image size: {image_size}\n')
f.write(f'train_batch_size: {train_batch_size}\n')
f.write(f'val_batch_size: {val_batch_size}\n')
f.write(f'test_batch_size: {test_batch_size}\n')
f.write('\n')


f.close()